Walk through all words given in a list and download the video associated. Warning a word may have several definition (some times almost the same), and here all will be downloaded. If you don't want that, use the other ("interactive") notebook.

In [1]:
import urllib
import os
from bs4 import BeautifulSoup
from IPython.core.display import HTML
import time

In [2]:
import unicodedata

def strip_accents(s):
       return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/nmondon/mots-frequents/master/frequence.csv", "frequence.csv")

('frequence.csv', <http.client.HTTPMessage at 0x10a86e0a0>)

In [4]:
words = []

with open("frequence.csv") as f:
    for line in f:
        cat, freq, word = line.strip().split(";")
        words.append((int(freq), word))

In [5]:
words = sorted(words, reverse=True)
words[:10]

[(1050561, 'le'),
 (862100, 'de'),
 (419564, 'un'),
 (362093, 'et'),
 (351960, 'être'),
 (270395, 'il'),
 (248488, 'avoir'),
 (186755, 'ne'),
 (184186, 'je'),
 (181161, 'son')]

In [6]:
uniq = set()
new_words = []
for w in words:
    w = w[1]
    n_w = strip_accents(w)
    if n_w not in uniq:
        new_words.append(w)
        uniq.add(n_w)

In [7]:
len(new_words), len(words)

(1357, 1461)

In [8]:
def crawl_page(word):
    page = urllib.request.urlopen("https://dico.elix-lsf.fr/dictionnaire/{}".format(word))
    return page

In [26]:
def crawl_word(word):
    page_url = "https://dico.elix-lsf.fr/dictionnaire/{}".format(word)
    page = urllib.request.urlopen(page_url)
    soup = BeautifulSoup(page)
    
    words = soup.find_all("section", {"class": "word"})
    #print(f"Found {len(words)} words")

    results = []
    for word in words:
        word_txt = word.find_all("h2", {"class": "word__title"})[0].text
        #print(f"# Word: {word_txt}")

        meanings = word.find_all("article", {"class": "meaning"})
        #print(f"\tFound {len(meanings)} meanings (taking only first one)")
        meaning = meanings[0]
        video = meaning.find_all("ul", {"class": "sign-list"})
        if len(video) == 0:
            continue
        video = video[0].findAll("video")
        if len(video) == 0:
            continue
        signs = meaning.find_all("ul", {"class": "sign-list"})
        #print(video[0]["src"], meaning.findAll("h3")[0].text)
        results.append({
            "word": word_txt, 
            "meaning": meaning.findAll("h3")[0].text, 
            "uri": video[0]["src"],
            "url": page_url
        })
        
        time.sleep(5.0)  # wait 5 seconds to avoid crushing Elix servers
    return results


def download_video(uri, path, folder="videos"):
    print(f"\t uri-> {uri}")
    video = urllib.request.urlopen(uri)
    tmp = video.read()
    with open(os.path.join(folder, path), "wb+") as f:
        f.write(tmp)
    

In [10]:
with open("cards.csv", "w+") as f:
    for i, tup in enumerate(new_words):
        print(f'======================= {i}/{len(words)}')

        word = tup
        word = strip_accents(word)
        try:
            results = crawl_word(word)
        except:
            print(f"-_______------___ Failed for {word}")
            continue
        for j in range(len(results)):
            norm_w = results[j]['word'].replace('(', '').replace(')', '').replace(' ', '_').replace('.', '')
            norm_w = strip_accents(norm_w)
            name = f"lsf_v2_{norm_w}_{j}.mp4"
            download_video(results[j]["uri"], name)
            results[j]["video"] = f"[sound:{name}]"
            
            w = results[j]['word']
            m = results[j]['meaning']
            url = results[j]['url']
            uri = results[j]['uri']
            video = results[j]['video']
            
            f.write(f'"{w}";"{m}";{url};{uri};{video}\n')

======================= 0/1461
======================= 1/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/de_nm_1_1.mp4
======================= 2/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/un_adj_1_1.mp4
	 uri-> https://www.elix-lsf.fr/IMG/mp4/un_det_1_1.mp4
======================= 3/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/et_cnj_1_1.mp4
======================= 4/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/etre_nm_1_2.mp4
	 uri-> https://www.elix-lsf.fr/IMG/mp4/etre_v_1_1.mp4
======================= 5/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/il.mp4
======================= 6/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/avoir_nm_1_1.mp4
	 uri-> https://www.elix-lsf.fr/IMG/mp4/avoir_v_1_1.mp4
======================= 7/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/naitre_v_1_1.mp4
	 uri-> https://www.elix-lsf.fr/IMG/mp4/ne_adj_1_1.mp4
======================= 8/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/je_pro_1_1.mp4
======================= 9/1461
	 uri-> https://www.elix-lsf.fr/I

	 uri-> https://www.elix-lsf.fr/IMG/mp4/autre_adj_1_1.mp4
======================= 89/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/apres_adv_1_1.mp4
	 uri-> https://www.elix-lsf.fr/IMG/mp4/apres_prp_1_1.mp4
======================= 90/1461
======================= 91/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/toujours_adv_1_1.mp4
======================= 92/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/00296.mp4
	 uri-> https://www.elix-lsf.fr/IMG/mp4/pouvoir_v_1_2.mp4
	 uri-> https://www.elix-lsf.fr/IMG/mp4/pouvoir_nm_1_2.mp4
======================= 93/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/jamais_adv_1_1.mp4
======================= 94/1461
======================= 95/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/aimer-2.mp4
======================= 96/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/non_adv_1_1.mp4
	 uri-> https://www.elix-lsf.fr/IMG/mp4/non_nm_1_1.mp4
======================= 97/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/heure_nf_1_1.mp4
======================= 98/1461

======================= 168/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/deja_adv_1_1.mp4
======================= 169/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/pied_n_1_1.mp4
	 uri-> https://www.elix-lsf.fr/IMG/mp4/pied_nm_1_2.mp4
======================= 170/1461
======================= 171/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/gens_nmp_1_1.mp4
======================= 172/1461
-_______------___ Failed for parce que
======================= 173/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/nom_nm_1_1.mp4
======================= 174/1461
======================= 175/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/entrer_v_1_1.mp4
======================= 176/1461
======================= 177/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/porter_v_1_2.mp4
======================= 178/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/00360.mp4
======================= 179/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/ciel_nm_1_6.mp4
======================= 180/1461
	 uri-> https://www.elix-lsf.fr/

======================= 260/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/video547-2.mp4
	 uri-> https://www.elix-lsf.fr/IMG/mp4/video547.mp4
======================= 261/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/fait_adj_1_1.mp4
	 uri-> https://www.elix-lsf.fr/IMG/mp4/fait_nm_1_2.mp4
======================= 262/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/parole_nf_1_1.mp4
======================= 263/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/guerre.mp4
======================= 264/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/tout_adj_1_1.mp4
	 uri-> https://www.elix-lsf.fr/IMG/mp4/tout_adv_1_1.mp4
	 uri-> https://www.elix-lsf.fr/IMG/mp4/tout_nm_1_1.mp4
======================= 265/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/ecouter_v_1_2.mp4
======================= 266/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/penser_nm_1_3.mp4
	 uri-> https://www.elix-lsf.fr/IMG/mp4/penser_v_1_1.mp4
======================= 267/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/affaire_nf_1_1_.mp4
	 u

	 uri-> https://www.elix-lsf.fr/IMG/mp4/un_adj_1_1.mp4
	 uri-> https://www.elix-lsf.fr/IMG/mp4/un_det_1_1.mp4
======================= 347/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/chacun_pro_1_1.mp4
======================= 348/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/profond_adj_1_1.mp4
	 uri-> https://www.elix-lsf.fr/IMG/mp4/profond_adv_1_1.mp4
======================= 349/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/argent_nm_1_1.mp4
======================= 350/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/cause_nf_1_1.mp4
	 uri-> https://www.elix-lsf.fr/IMG/mp4/video327-2.mp4
======================= 351/1461
======================= 352/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/autant_adv_1_1.mp4
======================= 353/1461
======================= 354/1461
======================= 355/1461
======================= 356/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/oeil_nm_1_1.mp4
======================= 357/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/dabord_adv_1_6.mp4
====

...

======================= 929/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/voile_nf_1_1.mp4
	 uri-> https://www.elix-lsf.fr/IMG/mp4/voile_nm_1_1.mp4
======================= 930/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/eclater_v_1_1.mp4
======================= 931/1461
======================= 932/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/quartier_n_1_2.mp4
	 uri-> https://www.elix-lsf.fr/IMG/mp4/quartier_nm_1_1.mp4
======================= 933/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/importance_nf_1_1.mp4
======================= 934/1461
======================= 935/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/religion_nf_1_1.mp4
======================= 936/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/reunir_v_1_1.mp4
======================= 937/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/palais_nm_1_1.mp4
======================= 938/1461
======================= 939/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/intelligence_nf_1_2.mp4
======================= 940/1461
============

======================= 1029/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/magnifique_adj_1_2.mp4
======================= 1030/1461
======================= 1031/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/temoin_nm_1_1.mp4
======================= 1032/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/visite_n_1_1.mp4
	 uri-> https://www.elix-lsf.fr/IMG/mp4/visiter_v_1_1.mp4
======================= 1033/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/respect_nm_1_1.mp4
======================= 1034/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/solitude.mp4
======================= 1035/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/video302.mp4
======================= 1036/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/prochain_adj_1_1.mp4
======================= 1037/1461
======================= 1038/1461
======================= 1039/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/anglais_adj_1_6.mp4
	 uri-> https://www.elix-lsf.fr/IMG/mp4/anglais_nprop_1_1.mp4
======================= 1040/1461
	 uri-> 

======================= 1133/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/presser_v_1_1.mp4
======================= 1134/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/effacer_v_1_1.mp4
======================= 1135/1461
======================= 1136/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/user_v_1_1.mp4
======================= 1137/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/reculer_v_1_1.mp4
======================= 1138/1461
======================= 1139/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/dangereux.mp4
======================= 1140/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/poesie_nf_1_1.mp4
======================= 1141/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/sommet_n_1_1.mp4
	 uri-> https://www.elix-lsf.fr/IMG/mp4/sommet_nm_1_1.mp4
======================= 1142/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/remplacer_v_1_1.mp4
======================= 1143/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/souhaiter_v_1_1.mp4
======================= 1144/1461
	 uri-> https://

	 uri-> https://www.elix-lsf.fr/IMG/mp4/renoncer_v_1_1.mp4
======================= 1243/1461
======================= 1244/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/extraordinaire_adj_1_1.mp4
	 uri-> https://www.elix-lsf.fr/IMG/mp4/extraordinaire_n_1_1.mp4
======================= 1245/1461
======================= 1246/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/transformer_v_1_1.mp4
======================= 1247/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/tracer_v_1_1.mp4
======================= 1248/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/chute_n_1_1.mp4
	 uri-> https://www.elix-lsf.fr/IMG/mp4/chute_nf_1_1.mp4
======================= 1249/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/divers_adj_1_11.mp4
======================= 1250/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/resistance-2.mp4
======================= 1251/1461
======================= 1252/1461
======================= 1253/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/mince_adj_1_1.mp4
======================= 1254

======================= 1347/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/professeur_nm_1_1.mp4
======================= 1348/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/perdre_v_1_1.mp4
======================= 1349/1461
======================= 1350/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/lien_nm_1_1.mp4
======================= 1351/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/queue_nf_1_1.mp4
======================= 1352/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/confondre_v_1_1.mp4
======================= 1353/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/bande.mp4
	 uri-> https://www.elix-lsf.fr/IMG/mp4/bander_v_1_1.mp4
======================= 1354/1461
======================= 1355/1461
	 uri-> https://www.elix-lsf.fr/IMG/mp4/mensonge.mp4
======================= 1356/1461
